In [1]:
from app import app, db, bcrypt
from models import User, Stock, Portfolio
from flask import jsonify, request
import yfinance as yf
import pandas as pd
import sqlite3

In [2]:
companyNames = 'HDFCBANK.NS HINDUNILVR.NS ASIANPAINT.NS RELIANCE.NS TCS.NS AUROPHARMA.NS BBTC.NS BALKRISIND.NS GARFIBRES.NS COLPAL.NS NESTLEIND.NS MARUTI.NS BRITANNIA.NS BOSCHLTD.NS ONGC.NS ABBOTINDIA.NS DIVISLAB.NS BAJFINANCE.NS RECLTD.NS PFC.NS PIDILITIND.NS ARE&M.NS'

In [4]:
companyNamesArray = companyNames.split() 

In [5]:
multipleStocks = yf.Tickers(companyNames)

In [6]:
ms_stockprices = multipleStocks.history(period="max")

[*********************100%***********************]  22 of 22 completed


In [7]:
ms_stockprices

Price              Close                                           \
Ticker     ABBOTINDIA.NS     ARE&M.NS ASIANPAINT.NS AUROPHARMA.NS   
Date                                                                
1996-01-01           NaN          NaN           NaN           NaN   
1996-01-02           NaN          NaN           NaN           NaN   
1996-01-03           NaN          NaN           NaN           NaN   
1996-01-04           NaN          NaN           NaN           NaN   
1996-01-05           NaN          NaN           NaN           NaN   
...                  ...          ...           ...           ...   
2025-01-20  27682.099609  1109.900024   2280.550049   1202.050049   
2025-01-21  27631.400391  1094.699951   2260.350098   1187.000000   
2025-01-22  27774.449219  1066.050049   2262.100098   1191.800049   
2025-01-23  27849.800781  1078.300049   2276.350098   1244.800049   
2025-01-24  27282.349609  1058.250000   2262.399902   1212.199951   

Price                                                              \
Ticker     BAJFINANCE.NS BALKRISIND.NS      BBTC.NS   BOSCHLTD.NS   
Date                                                                
1996-01-01           NaN           NaN          NaN           NaN   
1996-01-02           NaN           NaN          NaN           NaN   
1996-01-03           NaN           NaN          NaN           NaN   
1996-01-04           NaN           NaN          NaN           NaN   
1996-01-05           NaN           NaN          NaN           NaN   
...                  ...           ...          ...           ...   
2025-01-20   7440.100098   2721.500000  2034.699951  31136.750000   
2025-01-21   7291.649902   2681.600098  1980.099976  30611.000000   
2025-01-22   7397.149902   2708.449951  2230.149902  30582.699219   
2025-01-23   7443.250000   2728.899902  2232.199951  30990.500000   
2025-01-24   7438.600098   2652.199951  2161.949951  30356.500000   

Price                                 ...      Volume                          \
Ticker     BRITANNIA.NS    COLPAL.NS  ... HDFCBANK.NS HINDUNILVR.NS MARUTI.NS   
Date                                  ...                                       
1996-01-01          NaN          NaN  ...      350000         11000       NaN   
1996-01-02     7.473684          NaN  ...      412000        203500       NaN   
1996-01-03     7.379677          NaN  ...      284000         58000       NaN   
1996-01-04     7.379677          NaN  ...      282000        111500       NaN   
1996-01-05     7.384374          NaN  ...      189000         39500       NaN   
...                 ...          ...  ...         ...           ...       ...   
2025-01-20  4885.399902  2708.600098  ...     8778809       1382273  251731.0   
2025-01-21  4903.850098  2731.850098  ...    15579072       1842885  300038.0   
2025-01-22  4956.700195  2771.550049  ...    15309653       2658047  396590.0   
2025-01-23  5012.600098  2738.949951  ...    17454579       3408925  480876.0   
2025-01-24  5101.549805  2751.899902  ...    12118100       1688664  310343.0   

Price                                                                    \
Ticker     NESTLEIND.NS    ONGC.NS     PFC.NS PIDILITIND.NS   RECLTD.NS   
Date                                                                      
1996-01-01          NaN    10800.0        NaN           NaN         NaN   
1996-01-02          NaN    27000.0        NaN           NaN         NaN   
1996-01-03          NaN    10800.0        NaN           NaN         NaN   
1996-01-04          NaN    41400.0        NaN           NaN         NaN   
1996-01-05          NaN    46800.0        NaN           NaN         NaN   
...                 ...        ...        ...           ...         ...   
2025-01-20     401108.0  9676264.0  4692710.0      202462.0   5000936.0   
2025-01-21     764414.0  8989218.0  5673560.0      359612.0   5395660.0   
2025-01-22     448036.0  7919682.0  7990557.0      313441.0  10288459.0   
2025-01-23     496007.0  502667

In [8]:
stockData_22 = ms_stockprices.loc[:,(['Close','Volume','Dividends','Stock Splits'],companyNamesArray)]

In [9]:
stockData_22

Price             Close                                                        \
Ticker      HDFCBANK.NS HINDUNILVR.NS ASIANPAINT.NS  RELIANCE.NS       TCS.NS   
Date                                                                            
1996-01-01     2.331030     34.524017           NaN     3.654913          NaN   
1996-01-02     2.327118     34.892689           NaN     3.626490          NaN   
1996-01-03     2.334940     34.686008           NaN     3.654025          NaN   
1996-01-04     2.319296     34.688812           NaN     3.620273          NaN   
1996-01-05     2.315385     34.632938           NaN     3.595404          NaN   
...                 ...           ...           ...          ...          ...   
2025-01-20  1651.250000   2345.199951   2280.550049  1305.449951  4077.800049   
2025-01-21  1642.400024   2340.500000   2260.350098  1273.699951  4035.850098   
2025-01-22  1666.050049   2342.949951   2262.100098  1277.099976  4156.600098   
2025-01-23  1664.900024   2321.699951   2276.350098  1263.650024  4145.450195   
2025-01-24  1649.800049   2368.100098   2262.399902  1246.300049  4152.350098   

Price                                                                          \
Ticker     AUROPHARMA.NS      BBTC.NS BALKRISIND.NS GARFIBRES.NS    COLPAL.NS   
Date                                                                            
1996-01-01           NaN          NaN           NaN          NaN          NaN   
1996-01-02           NaN          NaN           NaN          NaN          NaN   
1996-01-03           NaN          NaN           NaN          NaN          NaN   
1996-01-04           NaN          NaN           NaN          NaN          NaN   
1996-01-05           NaN          NaN           NaN          NaN          NaN   
...                  ...          ...           ...          ...          ...   
2025-01-20   1202.050049  2034.699951   2721.500000   803.349976  2708.600098   
2025-01-21   1187.000000  1980.099976   2681.600098   778.000000  2731.850098   
2025-01-22   1191.800049  2230.149902   2708.449951   787.799988  2771.550049   
2025-01-23   1244.800049  2232.199951   2728.899902   809.150024  2738.949951   
2025-01-24   1212.199951  2161.949951   2652.199951   769.700012  2751.899902   

Price       ... Stock Splits                                                \
Ticker      ... BRITANNIA.NS BOSCHLTD.NS ONGC.NS ABBOTINDIA.NS DIVISLAB.NS   
Date        ...                                                              
1996-01-01  ...          NaN         NaN     0.0           NaN         NaN   
1996-01-02  ...          0.0         NaN     0.0           NaN         NaN   
1996-01-03  ...          0.0         NaN     0.0           NaN         NaN   
1996-01-04  ...          0.0         NaN     0.0           NaN         NaN   
1996-01-05  ...          0.0         NaN     0.0           NaN         NaN   
...         ...          ...         ...     ...           ...         ...   
2025-01-20  ...          0.0         0.0     0.0           0.0         0.0   
2025-01-21  ...          0.0         0.0     0.0           0.0         0.0   
2025-01-22  ...          0.0         0.0     0.0           0.0         0.0   
2025-01-23  ...          0.0         0.0     0.0           0.0         0.0   
2025-01-24  ...          0.0         0.0     0.0           0.0         0.0   

Price                                                             
Ticker     BAJFINANCE.NS RECLTD.NS PFC.NS PIDILITIND.NS ARE&M.NS  
Date                                                              
1996-01-01           NaN       NaN    NaN           NaN      NaN  
1996-01-02           NaN       NaN    NaN           NaN      NaN  
1996-01-03           NaN       NaN    NaN           NaN      NaN  
1996-01-04           NaN       NaN    NaN           NaN      NaN  
1996-01-05           NaN       NaN    NaN           NaN      NaN  
...                  ...       ...    ...           ...      ...  
2025-01-20           0.0       0.0

In [13]:
stockData_22.columns = stockData_22.columns.map(lambda x: f"{x[0]}_{x[1]}")

In [14]:
stockData_22

,Close_HDFCBANK.NS,Close_HINDUNILVR.NS,Close_ASIANPAINT.NS,Close_RELIANCE.NS,Close_TCS.NS,Close_AUROPHARMA.NS,Close_BBTC.NS,Close_BALKRISIND.NS,Close_GARFIBRES.NS,Close_COLPAL.NS,...,Stock Splits_BRITANNIA.NS,Stock Splits_BOSCHLTD.NS,Stock Splits_ONGC.NS,Stock Splits_ABBOTINDIA.NS,Stock Splits_DIVISLAB.NS,Stock Splits_BAJFINANCE.NS,Stock Splits_RECLTD.NS,Stock Splits_PFC.NS,Stock Splits_PIDILITIND.NS,Stock Splits_ARE&M.NS
Date,,,,,,,,,,,,,,,,,,,,,
1996-01-01,2.331030,34.524017,NaN,3.654913,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-02,2.327118,34.892689,NaN,3.626490,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-03,2.334940,34.686008,NaN,3.654025,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-04,2.319296,34.688812,NaN,3.620273,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-05,2.315385,34.632938,NaN,3.595404,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-20,1651.250000,2345.199951,2280.550049,1305.449951,4077.800049,1202.050049,2034.699951,2721.500000,803.349976,2708.600098,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-01-21,1642.400024,2340.500000,2260.350098,1273.699951,4035.850098,1187.000000,1980.099976,2681.600098,778.000000,2731.850098,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-01-22,1666.050049,2342.949951,2262.100098,1277.099976,4156.600098,1191.800049,2230.149902,2708.449951,787.799988,2771.550049,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
companyNamesArray

['HDFCBANK.NS',
 'HINDUNILVR.NS',
 'ASIANPAINT.NS',
 'RELIANCE.NS',
 'TCS.NS',
 'AUROPHARMA.NS',
 'BBTC.NS',
 'BALKRISIND.NS',
 'GARFIBRES.NS',
 'COLPAL.NS',
 'NESTLEIND.NS',
 'MARUTI.NS',
 'BRITANNIA.NS',
 'BOSCHLTD.NS',
 'ONGC.NS',
 'ABBOTINDIA.NS',
 'DIVISLAB.NS',
 'BAJFINANCE.NS',
 'RECLTD.NS',
 'PFC.NS',
 'PIDILITIND.NS',
 'ARE&M.NS']

In [16]:
metrics = ['Close','Volume', 'Dividends', 'Stock Splits']

In [17]:
# Prepare the value_vars dictionary dynamically
value_vars = {}
for company in companyNamesArray:
    for metric in metrics:
        column_name = f"{metric}_{company}"
        if column_name in stockData_22.columns:  # Check if the column exists
            value_vars[column_name] = (company, metric)

In [18]:
value_vars

{'Close_HDFCBANK.NS': ('HDFCBANK.NS', 'Close'),
 'Volume_HDFCBANK.NS': ('HDFCBANK.NS', 'Volume'),
 'Dividends_HDFCBANK.NS': ('HDFCBANK.NS', 'Dividends'),
 'Stock Splits_HDFCBANK.NS': ('HDFCBANK.NS', 'Stock Splits'),
 'Close_HINDUNILVR.NS': ('HINDUNILVR.NS', 'Close'),
 'Volume_HINDUNILVR.NS': ('HINDUNILVR.NS', 'Volume'),
 'Dividends_HINDUNILVR.NS': ('HINDUNILVR.NS', 'Dividends'),
 'Stock Splits_HINDUNILVR.NS': ('HINDUNILVR.NS', 'Stock Splits'),
 'Close_ASIANPAINT.NS': ('ASIANPAINT.NS', 'Close'),
 'Volume_ASIANPAINT.NS': ('ASIANPAINT.NS', 'Volume'),
 'Dividends_ASIANPAINT.NS': ('ASIANPAINT.NS', 'Dividends'),
 'Stock Splits_ASIANPAINT.NS': ('ASIANPAINT.NS', 'Stock Splits'),
 'Close_RELIANCE.NS': ('RELIANCE.NS', 'Close'),
 'Volume_RELIANCE.NS': ('RELIANCE.NS', 'Volume'),
 'Dividends_RELIANCE.NS': ('RELIANCE.NS', 'Dividends'),
 'Stock Splits_RELIANCE.NS': ('RELIANCE.NS', 'Stock Splits'),
 'Close_TCS.NS': ('TCS.NS', 'Close'),
 'Volume_TCS.NS': ('TCS.NS', 'Volume'),
 'Dividends_TCS.NS': ('TCS

In [19]:
stockData_22=stockData_22.reset_index()

In [20]:
stockData_22

,Date,Close_HDFCBANK.NS,Close_HINDUNILVR.NS,Close_ASIANPAINT.NS,Close_RELIANCE.NS,Close_TCS.NS,Close_AUROPHARMA.NS,Close_BBTC.NS,Close_BALKRISIND.NS,Close_GARFIBRES.NS,...,Stock Splits_BRITANNIA.NS,Stock Splits_BOSCHLTD.NS,Stock Splits_ONGC.NS,Stock Splits_ABBOTINDIA.NS,Stock Splits_DIVISLAB.NS,Stock Splits_BAJFINANCE.NS,Stock Splits_RECLTD.NS,Stock Splits_PFC.NS,Stock Splits_PIDILITIND.NS,Stock Splits_ARE&M.NS
0,1996-01-01,2.331030,34.524017,NaN,3.654913,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1996-01-02,2.327118,34.892689,NaN,3.626490,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1996-01-03,2.334940,34.686008,NaN,3.654025,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1996-01-04,2.319296,34.688812,NaN,3.620273,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1996-01-05,2.315385,34.632938,NaN,3.595404,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7297,2025-01-20,1651.250000,2345.199951,2280.550049,1305.449951,4077.800049,1202.050049,2034.699951,2721.500000,803.349976,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7298,2025-01-21,1642.400024,2340.500000,2260.350098,1273.699951,4035.850098,1187.000000,1980.099976,2681.600098,778.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7299,2025-01-22,1666.050049,2342.949951,2262.100098,1277.099976,4156.600098,1191.800049,2230.149902,2708.449951,787.799988,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7300,2025-01-23,1664.900024,2321.699951,2276.350098,1263.650024,4145.450195,1244.800049,2232.199951,2728.899902,809.150024,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
df_long = stockData_22.melt(
    id_vars=["Date"], 
    var_name='variable',  # Use a scalar here
    value_name='value',
    value_vars=value_vars
)

In [22]:
df_long

,Date,variable,value
0,1996-01-01,Close_HDFCBANK.NS,2.331030
1,1996-01-02,Close_HDFCBANK.NS,2.327118
2,1996-01-03,Close_HDFCBANK.NS,2.334940
3,1996-01-04,Close_HDFCBANK.NS,2.319296
4,1996-01-05,Close_HDFCBANK.NS,2.315385
...,...,...,...
642571,2025-01-20,Stock Splits_ARE&M.NS,0.000000
642572,2025-01-21,Stock Splits_ARE&M.NS,0.000000
642573,2025-01-22,Stock Splits_ARE&M.NS,0.000000
642574,2025-01-23,Stock Splits_ARE&M.NS,0.000000


In [23]:
# Split the 'variable' column into 'company' and 'metric'
df_long[['metric','company']] = df_long['variable'].str.split('_', expand=True)

In [24]:
df_long

,Date,variable,value,metric,company
0,1996-01-01,Close_HDFCBANK.NS,2.331030,Close,HDFCBANK.NS
1,1996-01-02,Close_HDFCBANK.NS,2.327118,Close,HDFCBANK.NS
2,1996-01-03,Close_HDFCBANK.NS,2.334940,Close,HDFCBANK.NS
3,1996-01-04,Close_HDFCBANK.NS,2.319296,Close,HDFCBANK.NS
4,1996-01-05,Close_HDFCBANK.NS,2.315385,Close,HDFCBANK.NS
...,...,...,...,...,...
642571,2025-01-20,Stock Splits_ARE&M.NS,0.000000,Stock Splits,ARE&M.NS
642572,2025-01-21,Stock Splits_ARE&M.NS,0.000000,Stock Splits,ARE&M.NS
642573,2025-01-22,Stock Splits_ARE&M.NS,0.000000,Stock Splits,ARE&M.NS
642574,2025-01-23,Stock Splits_ARE&M.NS,0.000000,Stock Splits,ARE&M.NS


In [25]:
df_long = df_long.drop(columns='variable')

In [26]:
df_long

,Date,value,metric,company
0,1996-01-01,2.331030,Close,HDFCBANK.NS
1,1996-01-02,2.327118,Close,HDFCBANK.NS
2,1996-01-03,2.334940,Close,HDFCBANK.NS
3,1996-01-04,2.319296,Close,HDFCBANK.NS
4,1996-01-05,2.315385,Close,HDFCBANK.NS
...,...,...,...,...
642571,2025-01-20,0.000000,Stock Splits,ARE&M.NS
642572,2025-01-21,0.000000,Stock Splits,ARE&M.NS
642573,2025-01-22,0.000000,Stock Splits,ARE&M.NS
642574,2025-01-23,0.000000,Stock Splits,ARE&M.NS


In [27]:
new_order = ['Date','company','metric','value']

In [28]:
df_long=df_long[new_order]

In [29]:
df_long

,Date,company,metric,value
0,1996-01-01,HDFCBANK.NS,Close,2.331030
1,1996-01-02,HDFCBANK.NS,Close,2.327118
2,1996-01-03,HDFCBANK.NS,Close,2.334940
3,1996-01-04,HDFCBANK.NS,Close,2.319296
4,1996-01-05,HDFCBANK.NS,Close,2.315385
...,...,...,...,...
642571,2025-01-20,ARE&M.NS,Stock Splits,0.000000
642572,2025-01-21,ARE&M.NS,Stock Splits,0.000000
642573,2025-01-22,ARE&M.NS,Stock Splits,0.000000
642574,2025-01-23,ARE&M.NS,Stock Splits,0.000000


In [31]:
# Create Metrics Table
metrics = df_long[['metric']].drop_duplicates().reset_index(drop=True)
metrics['Metric_ID'] = metrics.index + 1
metrics = metrics[['Metric_ID', 'metric']]

In [32]:
metrics

,Metric_ID,metric
0,1,Close
1,2,Volume
2,3,Dividends
3,4,Stock Splits


In [33]:
# Create Companies Table
companies = df_long[['company']].drop_duplicates().reset_index(drop=True)
companies['Company_ID'] = companies.index + 1
companies = companies[['Company_ID', 'company']]

In [34]:
companies

,Company_ID,company
0,1,HDFCBANK.NS
1,2,HINDUNILVR.NS
2,3,ASIANPAINT.NS
3,4,RELIANCE.NS
4,5,TCS.NS
5,6,AUROPHARMA.NS
6,7,BBTC.NS
7,8,BALKRISIND.NS
8,9,GARFIBRES.NS
9,10,COLPAL.NS


In [35]:
# Create Data Table
data = df_long.merge(companies, on='company').merge(metrics, on='metric')

In [36]:
data

,Date,company,metric,value,Company_ID,Metric_ID
0,1996-01-01,HDFCBANK.NS,Close,2.331030,1,1
1,1996-01-02,HDFCBANK.NS,Close,2.327118,1,1
2,1996-01-03,HDFCBANK.NS,Close,2.334940,1,1
3,1996-01-04,HDFCBANK.NS,Close,2.319296,1,1
4,1996-01-05,HDFCBANK.NS,Close,2.315385,1,1
...,...,...,...,...,...,...
642571,2025-01-20,ARE&M.NS,Stock Splits,0.000000,22,4
642572,2025-01-21,ARE&M.NS,Stock Splits,0.000000,22,4
642573,2025-01-22,ARE&M.NS,Stock Splits,0.000000,22,4
642574,2025-01-23,ARE&M.NS,Stock Splits,0.000000,22,4


In [38]:
data = data[['Date', 'Company_ID', 'Metric_ID', 'value']]

In [39]:
data

,Date,Company_ID,Metric_ID,value
0,1996-01-01,1,1,2.331030
1,1996-01-02,1,1,2.327118
2,1996-01-03,1,1,2.334940
3,1996-01-04,1,1,2.319296
4,1996-01-05,1,1,2.315385
...,...,...,...,...
642571,2025-01-20,22,4,0.000000
642572,2025-01-21,22,4,0.000000
642573,2025-01-22,22,4,0.000000
642574,2025-01-23,22,4,0.000000


In [40]:
metrics

,Metric_ID,metric
0,1,Close
1,2,Volume
2,3,Dividends
3,4,Stock Splits


In [41]:
companies

,Company_ID,company
0,1,HDFCBANK.NS
1,2,HINDUNILVR.NS
2,3,ASIANPAINT.NS
3,4,RELIANCE.NS
4,5,TCS.NS
5,6,AUROPHARMA.NS
6,7,BBTC.NS
7,8,BALKRISIND.NS
8,9,GARFIBRES.NS
9,10,COLPAL.NS


In [42]:
# Connect to SQLite database (will create one if it doesn't exist)
conn = sqlite3.connect('stock_data.db')

# Write the dataframes to SQL tables
metrics.to_sql('Metrics', conn, if_exists='replace', index=False)
companies.to_sql('Companies', conn, if_exists='replace', index=False)
data.to_sql('Data', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

In [45]:
# Reconnect to the database
conn = sqlite3.connect('stock_data.db')
cursor = conn.cursor()

In [46]:
# Example Query 1: Get all data for Company_A on a specific date
query1 = """
SELECT d.Date, c.Company, m.Metric, d.Value 
FROM Data d
JOIN Companies c ON d.Company_ID = c.Company_ID
JOIN Metrics m ON d.Metric_ID = m.Metric_ID
WHERE c.company = 'HDFCBANK.NS' 
  AND d.Date = '2023-01-01';
"""
cursor.execute(query1)
for row in cursor.fetchall():
    print(row)

In [48]:
cursor.fetchall()

[]

In [47]:
query1

"\nSELECT d.Date, c.Company, m.Metric, d.Value \nFROM Data d\nJOIN Companies c ON d.Company_ID = c.Company_ID\nJOIN Metrics m ON d.Metric_ID = m.Metric_ID\nWHERE c.company = 'HDFCBANK.NS' \n  AND d.Date = '2023-01-01';\n"

In [51]:
# Example Query 1: Get all data for Company_A on a specific date
query1 = """
SELECT *
FROM Data
WHERE Company_ID =2;
"""
cursor.execute(query1)
for row in cursor.fetchall():
    print(row)

('1996-01-01 00:00:00', 2, 1, 34.524017333984375)
('1996-01-02 00:00:00', 2, 1, 34.8926887512207)
('1996-01-03 00:00:00', 2, 1, 34.68600845336914)
('1996-01-04 00:00:00', 2, 1, 34.688812255859375)
('1996-01-05 00:00:00', 2, 1, 34.632938385009766)
('1996-01-08 00:00:00', 2, 1, 34.64691162109375)
('1996-01-09 00:00:00', 2, 1, 34.77817916870117)
('1996-01-10 00:00:00', 2, 1, 35.02674865722656)
('1996-01-11 00:00:00', 2, 1, 34.85639190673828)
('1996-01-12 00:00:00', 2, 1, 34.87593078613281)
('1996-01-15 00:00:00', 2, 1, 34.85639190673828)
('1996-01-16 00:00:00', 2, 1, 34.987648010253906)
('1996-01-17 00:00:00', 2, 1, 35.14963150024414)
('1996-01-18 00:00:00', 2, 1, 35.1915283203125)
('1996-01-19 00:00:00', 2, 1, 35.57137680053711)
('1996-01-22 00:00:00', 2, 1, 35.50435256958008)
('1996-01-23 00:00:00', 2, 1, 35.554622650146484)
('1996-01-24 00:00:00', 2, 1, 35.3814697265625)
('1996-01-25 00:00:00', 2, 1, 35.35911560058594)
('1996-01-26 00:00:00', 2, 1, 35.35911560058594)
('1996-01-29 00:00

In [52]:
# Close the cursor and connection
cursor.close()
conn.close()